## Introduction

Youtube is one of the most popular video sharing platform with more than 1 billion users. Users have long been outraged by the overwhelming number of spam messages in the comment section. In 2012 users created a petition asking Youtube to provide tools to deal with undesired content. In 2013, spam problem gets worse as Google overhauled the YouTube comment system to connect it to Google+, which allows users to post links. This attracts more malicious users to self-promote their videos using the platform. This project will build a spam filter to automatically filter spam comments.



## Data Wrangling

In this part we will:

    1: load the data into notebook
    2: check for any missing values

In [1]:
import pandas as pd


In [2]:
s1=pd.read_csv('./data/Youtube01-Psy.csv',encoding = 'utf-8-sig')
s2=pd.read_csv('./data/Youtube02-KatyPerry.csv',encoding = 'utf-8-sig')
s3=pd.read_csv('./data/Youtube03-LMFAO.csv',encoding = 'utf-8-sig')
s4=pd.read_csv('./data/Youtube04-Eminem.csv',encoding = 'utf-8-sig')
s5=pd.read_csv('./data/Youtube05-Shakira.csv',encoding = 'utf-8-sig')
s1['song']='Psy'
s2['song']='KetyPerry'
s3['song']='LMFAO'
s4['song']='Eminem'
s5['song']='Shakira'

In [3]:
df=pd.concat([s1,s2,s3,s4,s5])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1956 entries, 0 to 369
Data columns (total 6 columns):
COMMENT_ID    1956 non-null object
AUTHOR        1956 non-null object
DATE          1711 non-null object
CONTENT       1956 non-null object
CLASS         1956 non-null int64
song          1956 non-null object
dtypes: int64(1), object(5)
memory usage: 107.0+ KB


In [4]:
#df.groupby('AUTHOR').count().sort_values('song',ascending=False)

In [5]:
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,song
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1,Psy
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1,Psy
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1,Psy
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1,Psy
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1,Psy



There are total of 1956 comments in this data for the 5 most popular songs. There are some missing values in the column of date, but we will focus mostly on the CONTENT column so this will not affect us. The CLASS section is the label: 1 means spam and 0 means ham. 


## Data Analysis

In [6]:
## check the length of the comment.

df['CONTENT'].iloc[100]

'My videos are half way decent, check them out if you want.\ufeff'

In [7]:
import contractions

def contraction_expand(text):
    return contractions.fix(text)

df['CONTENT']=df['CONTENT'].apply(contraction_expand)

In [8]:

#tokenize workds

import nltk
df['words']=df['CONTENT'].apply(nltk.word_tokenize)

#calculate sentence length

df['length']=df['CONTENT'].apply(len)

df.groupby('CLASS')['length'].agg(['mean','std','min','max'])

,mean,std,min,max
CLASS,,,,
0,49.827550,56.526731,2,755
1,137.769154,159.459172,10,1200


In [9]:
## check if there contains URL

import re

def URL (string):
    pattern=r'http[s]?://*'
    return (bool(re.search(pattern,string)))

df['URL']=df['CONTENT'].apply(URL)
sum(df['URL'])/len(df['URL'])
#df.head()

0.10071574642126789

In [10]:

df.groupby('CLASS')['URL'].agg(sum)

CLASS
0     11.0
1    186.0
Name: URL, dtype: float64

In [11]:
# calculate number of capital alpabat

def capital_letters(string):
    return (len(re.findall('[A-Z]',string)))

df['capital']=df['CONTENT'].apply(capital_letters)

In [12]:
df.groupby('CLASS')['capital'].agg(['mean','std','min','max'])

,mean,std,min,max
CLASS,,,,
0,3.446898,7.497303,0,82
1,14.386070,34.527145,0,351


In [13]:

#remove non ASIC words
#lower all words
#remove stop words
#word count

def remove_non_ASIC (words):
   """remove non ASIC words"""
    
